In [1]:
from IPython.display import Image
from pathlib import Path
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import log_loss
import math

In [2]:
train_labels = pd.read_csv("KD/train_labels_KD.csv", index_col="seq_id")
c = ['seq_id'] + ['pred_' + i for i in list(train_labels.columns)] + ['x1'] + ['pred2_' + i for i in list(train_labels.columns)] + ['x2'] +['pred3_' + i for i in list(train_labels.columns)] +['x3'] + ['backgroung', 'img_count']
pred_labels = pd.read_csv("KD/insres_SER_S9.csv", index_col="seq_id", names=c)
print(len(pred_labels))
c = ['seq_id'] + ['xepred_' + i for i in list(train_labels.columns)] + ['x1'] + ['xepred2_' + i for i in list(train_labels.columns)] + ['x2'] +['xepred3_' + i for i in list(train_labels.columns)] +['x3'] + ['xebackgroung', 'xeimg_count']
pred_labels_ce = pd.read_csv("KD/Xception_plt_lr2_SER_S9.csv", index_col="seq_id", names=c)
pred_labels_ce=pred_labels_ce.iloc[:len(pred_labels)]

# c = ['seq_id'] + ['effpred_' + i for i in list(train_labels.columns)] + ['x1'] + ['effpred2_' + i for i in list(train_labels.columns)] + ['x2'] +['effpred3_' + i for i in list(train_labels.columns)] +['x3'] + ['effbackgroung', 'effimg_count']
# pred_labels_eff = pd.read_csv("KD/B3_all_KD_SER_S9_.csv", index_col="seq_id", names=c).iloc[:70000]
# pred_labels_eff=pred_labels_eff.iloc[:len(pred_labels)]


del pred_labels['x1']
del pred_labels['x2']
del pred_labels['x3']

del pred_labels_ce['x1']
del pred_labels_ce['x2']
del pred_labels_ce['x3']

# del pred_labels_eff['x1']
# del pred_labels_eff['x2']
# del pred_labels_eff['x3']

367463


In [3]:
len(pred_labels), len(pred_labels_ce)

(367463, 367463)

In [4]:
join = pred_labels.join(train_labels, how='left')

In [5]:
join = join.join(pred_labels_ce, how='left')

In [6]:
# join = join.join(pred_labels_eff, how='left')

In [7]:
DATA_PATH = Path("../../datasets/wildlife") 
train_metadata = pd.read_csv(DATA_PATH / "train_metadata.csv", parse_dates= ['datetime'], index_col="seq_id")
train_metadata = train_metadata.groupby('seq_id').first()
train_metadata['month']=train_metadata.datetime.dt.month
train_metadata['hour']=train_metadata.datetime.dt.hour
join = join.join(train_metadata.hour, how='left')
join = join.join(train_metadata.month, how='left')

In [8]:
join.loc[join.img_count<3, ['pred3_' + c for c in train_labels.columns]]=-0.01
join.loc[join.img_count<2, ['pred2_' + c for c in train_labels.columns]]=-0.01
join.loc[join.img_count<3, ['xepred3_' + c for c in train_labels.columns]]=-0.01
join.loc[join.img_count<2, ['xepred2_' + c for c in train_labels.columns]]=-0.01
join.loc[join.img_count<2, ['xebackgroung']]=-0.01
join.loc[join.img_count<2, ['backgroung']]=-0.01

In [9]:
# for c in list(train_labels.columns):
#     join['median_' + c] = join[['pred_' + c, 'pred2_' + c, 'pred3_' + c, 'xepred_' + c, 'xepred2_' + c, 'xepred3_' + c]].median(axis = 1)

In [29]:
test_dataset=join.copy()

In [38]:

test_index = set(join.index[::10])
test_dataset     = join[ join.index.isin(test_index)]
trainval_dataset = join[~join.index.isin(test_index)]

val_index = set(trainval_dataset.index[::8])
val_dataset   = trainval_dataset[ trainval_dataset.index.isin(val_index)]
train_dataset = trainval_dataset[~trainval_dataset.index.isin(val_index)]
animal_columns =list(train_labels.columns)
animal_columns.remove('empty')



In [35]:
# for c in list(train_labels.columns):
#     test_dataset['pred_' + c] = (test_dataset['pred_' + c] + test_dataset['xepred_' + c]*0.1)/1.1
#     test_dataset['pred2_' + c] = (test_dataset['pred2_' + c] + test_dataset['xepred2_' + c]*0.1)/1.1
#     test_dataset['pred3_' + c] = (test_dataset['pred3_' + c] + test_dataset['xepred3_' + c]*0.1)/1.1
#     test_dataset.loc[test_dataset.img_count < 3, 'pred3_' + c]=-999
#     test_dataset.loc[test_dataset.img_count < 2, 'pred2_' + c]=-999
#     test_dataset.loc[test_dataset.img_count == 2, 'pred_' + c] = (test_dataset.loc[test_dataset.img_count == 2, 'pred_' + c] +
#                                                   test_dataset.loc[test_dataset.img_count == 2, 'pred2_' + c]*0.9) /1.9
#     test_dataset.loc[test_dataset.img_count > 2, 'pred_' + c] = (test_dataset.loc[test_dataset.img_count > 2, 'pred_' + c] +
#                                                  test_dataset.loc[test_dataset.img_count > 2, 'pred2_' +c]*0.8 +
#                                                  test_dataset.loc[test_dataset.img_count > 2, 'pred3_' + c]*0.5) / 2.3


# test_dataset['backgroung']=test_dataset['backgroung']*0.5+test_dataset['xebackgroung']*0.5
# test_dataset.loc[test_dataset.img_count==2,'pred_empty'] = (test_dataset.loc[test_dataset.img_count==2,'pred_empty']*0.6 +test_dataset.loc[test_dataset.img_count==2,'backgroung']*0.4)
# test_dataset.loc[test_dataset.img_count>2,'pred_empty'] = (test_dataset.loc[test_dataset.img_count>2,'pred_empty']*0.1 +test_dataset.loc[test_dataset.img_count>2,'backgroung']*0.9)


/home/ivan/projects/wildlife/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ivan/projects/wildlife/venv/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ivan/projects/wildlife/venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

In [36]:
len(test_dataset)

36747

In [37]:
from sklearn.metrics import log_loss
loss = 0
for c in list(train_labels.columns):
    x = test_dataset[['pred_' + c, c]].values
    if sum(x[:,1]) !=0:
        l = log_loss(x[:,1], x[:,0])
        loss+=l
        print(c, l)
print(loss/54)
# 0.00659760884180274 img1
# 0.006059572723197153 img123
# 0.0045413404802776185 img123 with bg
# 0.0045249732546423236 img123 with bg + magic

# 0.003984543177212998

aardvark 0.00012919119132652598
aardwolf 0.00044935732611868723
baboon 0.0017437560784325328
batearedfox 0.0005621441629822682
buffalo 0.004173479149098593
bushbuck 0.00013517548068604142
caracal 0.00012829743237578866
cheetah 0.001481605348065354
civet 3.2090208981803474e-05
dikdik 0.001772843544694549
duiker 0.0009040750235482586
eland 0.001413437734057415
elephant 0.004275882656133998
empty 0.05413436299019591
gazellegrants 0.0072523063736520685
gazellethomsons 0.016724819842948404
genet 0.000293372497063182
giraffe 0.0027756507167913916
guineafowl 0.0013192404815288684
hare 0.00047963944873433557
hartebeest 0.002755468853549346
hippopotamus 0.0006632442384571759
honeybadger 0.000107907023180374
hyenaspotted 0.002719404127492644
hyenastriped 0.00021765209045491597
impala 0.004708035760708407
insectspider 0.0037726495712729065
jackal 0.0010182169314706358
koribustard 0.0009280341552816003
leopard 0.0004108911689607214
lionfemale 0.0021211899571976635
lionmale 0.0012469270357494855
mo

In [39]:
classes = ['gazellethomsons', 'empty', 'zebra', 'wildebeest',  'giraffe', 'elephant', 'insectspider'] 
feature_classes=list(train_labels.columns)
# feature_classes.remove('civet')
# feature_classes.remove('bat')
# feature_classes.remove('reptiles')
# feature_classes.remove('honeybadger')

# feature_classes=[ 'monkeyvervet']


num_leaves={
    'empty': 20,
    'elephant':10,
    'gazellegrants':10,
    
}
max_depth={
    'empty': -1,
    'elephant': -1,
    'gazellegrants': -1,
}
learning_rate={
    'empty':0.01,
    'otherbird':0.001,
    'gazellegrants':0.001,
}
# columns = ['pred_' + c for c in feature_classes] +\
# ['pred2_' + c for c in feature_classes] + \
# ['pred3_' + c for c in feature_classes] + \
# ['xebackgroung', 'backgroung', 'img_count']  +\
# ['hour', 'month']

columns = ['pred_' + c for c in feature_classes] +\
['xepred_' + c for c in feature_classes] + \
['pred2_' + c for c in feature_classes] + \
['xepred2_' + c for c in feature_classes] + \
['pred3_' + c for c in feature_classes] + \
['xepred3_' + c for c in feature_classes] + \
['xebackgroung', 'backgroung', 'img_count']  +\
['hour', 'month']
# columns = ['pred_' + c for c in classes] + ['pred2_' + c for c in classes] + ['pred3_' + c for c in classes] + ['backgroung', 'img_count', 'hour', 'month']


In [51]:
clazz='hare'
X_train = train_dataset[columns]
y_train = train_dataset[clazz]
X_val = val_dataset[columns]
y_val = val_dataset[clazz]

In [52]:
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# clf = GaussianNB()
# clf.fit(X_train, y_train)
# X_test = test_dataset[columns]
# y_test = test_dataset[clazz]

# print(clazz, log_loss(y_test, clf.predict_proba(X_test.values)[:,1]))

In [53]:


lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

scale = y_train.mean()
if scale < 0.5:
    scale=math.log(1/scale)
else:
    scale=math.sqrt(scale)
print(scale)
params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': {'binary_logloss'},
        'num_leaves': num_leaves.get(clazz, 5),
        'max_depth': max_depth.get(clazz, 3),
        'learning_rate': learning_rate.get(clazz, 0.01/scale),
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0,
#         'scale_pos_weight':scale_pos_weight.get(clazz, scale)
        'scale_pos_weight':1

}


gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=10000,
                    valid_sets=lgb_val,
                    early_stopping_rounds=50)
gbm.save_model("lgbs/" + clazz + '.txt', num_iteration=gbm.best_iteration) 

X_test = test_dataset[columns]
y_test = test_dataset[clazz]

print(clazz, log_loss(y_test, gbm.predict(X_test.values)))
# zebra 0.014600679685758957

7.190987096751118
[1]	valid_0's binary_logloss: 0.00627321
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.00539372
[3]	valid_0's binary_logloss: 0.00522213
[4]	valid_0's binary_logloss: 0.0050988
[5]	valid_0's binary_logloss: 0.00495705
[6]	valid_0's binary_logloss: 0.00483253
[7]	valid_0's binary_logloss: 0.00473147
[8]	valid_0's binary_logloss: 0.00464042
[9]	valid_0's binary_logloss: 0.00455841
[10]	valid_0's binary_logloss: 0.00449838
[11]	valid_0's binary_logloss: 0.00443074
[12]	valid_0's binary_logloss: 0.00436658
[13]	valid_0's binary_logloss: 0.00430928
[14]	valid_0's binary_logloss: 0.0042538
[15]	valid_0's binary_logloss: 0.00420228
[16]	valid_0's binary_logloss: 0.00415202
[17]	valid_0's binary_logloss: 0.00411117
[18]	valid_0's binary_logloss: 0.00406682
[19]	valid_0's binary_logloss: 0.00402425
[20]	valid_0's binary_logloss: 0.00398435
[21]	valid_0's binary_logloss: 0.00394744
[22]	valid_0's binary_logloss: 0.00391767
[23]	val

[196]	valid_0's binary_logloss: 0.00212787
[197]	valid_0's binary_logloss: 0.00212292
[198]	valid_0's binary_logloss: 0.00211801
[199]	valid_0's binary_logloss: 0.00211313
[200]	valid_0's binary_logloss: 0.00210865
[201]	valid_0's binary_logloss: 0.00210422
[202]	valid_0's binary_logloss: 0.00209982
[203]	valid_0's binary_logloss: 0.00209544
[204]	valid_0's binary_logloss: 0.00209121
[205]	valid_0's binary_logloss: 0.00208687
[206]	valid_0's binary_logloss: 0.00208269
[207]	valid_0's binary_logloss: 0.00207854
[208]	valid_0's binary_logloss: 0.00207439
[209]	valid_0's binary_logloss: 0.00207027
[210]	valid_0's binary_logloss: 0.00206617
[211]	valid_0's binary_logloss: 0.00206203
[212]	valid_0's binary_logloss: 0.00205821
[213]	valid_0's binary_logloss: 0.0020541
[214]	valid_0's binary_logloss: 0.00205002
[215]	valid_0's binary_logloss: 0.00204612
[216]	valid_0's binary_logloss: 0.00204209
[217]	valid_0's binary_logloss: 0.00203592
[218]	valid_0's binary_logloss: 0.00202994
[219]	valid_

[393]	valid_0's binary_logloss: 0.00150343
[394]	valid_0's binary_logloss: 0.00150125
[395]	valid_0's binary_logloss: 0.00149908
[396]	valid_0's binary_logloss: 0.00149699
[397]	valid_0's binary_logloss: 0.0014949
[398]	valid_0's binary_logloss: 0.00149283
[399]	valid_0's binary_logloss: 0.00149075
[400]	valid_0's binary_logloss: 0.00148857
[401]	valid_0's binary_logloss: 0.00148645
[402]	valid_0's binary_logloss: 0.00148439
[403]	valid_0's binary_logloss: 0.00148234
[404]	valid_0's binary_logloss: 0.0014803
[405]	valid_0's binary_logloss: 0.00147826
[406]	valid_0's binary_logloss: 0.00147619
[407]	valid_0's binary_logloss: 0.00147414
[408]	valid_0's binary_logloss: 0.00147209
[409]	valid_0's binary_logloss: 0.00147005
[410]	valid_0's binary_logloss: 0.00146801
[411]	valid_0's binary_logloss: 0.00146593
[412]	valid_0's binary_logloss: 0.00146386
[413]	valid_0's binary_logloss: 0.00146184
[414]	valid_0's binary_logloss: 0.00145948
[415]	valid_0's binary_logloss: 0.00145714
[416]	valid_0

[590]	valid_0's binary_logloss: 0.00116335
[591]	valid_0's binary_logloss: 0.00116172
[592]	valid_0's binary_logloss: 0.0011601
[593]	valid_0's binary_logloss: 0.00115874
[594]	valid_0's binary_logloss: 0.00115739
[595]	valid_0's binary_logloss: 0.00115578
[596]	valid_0's binary_logloss: 0.00115436
[597]	valid_0's binary_logloss: 0.00115304
[598]	valid_0's binary_logloss: 0.00115173
[599]	valid_0's binary_logloss: 0.00115042
[600]	valid_0's binary_logloss: 0.00114911
[601]	valid_0's binary_logloss: 0.00114777
[602]	valid_0's binary_logloss: 0.00114643
[603]	valid_0's binary_logloss: 0.00114509
[604]	valid_0's binary_logloss: 0.00114376
[605]	valid_0's binary_logloss: 0.00114243
[606]	valid_0's binary_logloss: 0.00114113
[607]	valid_0's binary_logloss: 0.00113984
[608]	valid_0's binary_logloss: 0.00113856
[609]	valid_0's binary_logloss: 0.00113728
[610]	valid_0's binary_logloss: 0.00113601
[611]	valid_0's binary_logloss: 0.00113465
[612]	valid_0's binary_logloss: 0.00113329
[613]	valid_

[781]	valid_0's binary_logloss: 0.000946522
[782]	valid_0's binary_logloss: 0.000945616
[783]	valid_0's binary_logloss: 0.000944711
[784]	valid_0's binary_logloss: 0.000943809
[785]	valid_0's binary_logloss: 0.000942909
[786]	valid_0's binary_logloss: 0.000941767
[787]	valid_0's binary_logloss: 0.000940843
[788]	valid_0's binary_logloss: 0.000939869
[789]	valid_0's binary_logloss: 0.000938948
[790]	valid_0's binary_logloss: 0.000938028
[791]	valid_0's binary_logloss: 0.000937133
[792]	valid_0's binary_logloss: 0.000936244
[793]	valid_0's binary_logloss: 0.000935356
[794]	valid_0's binary_logloss: 0.00093447
[795]	valid_0's binary_logloss: 0.000933581
[796]	valid_0's binary_logloss: 0.000932719
[797]	valid_0's binary_logloss: 0.00093201
[798]	valid_0's binary_logloss: 0.000931152
[799]	valid_0's binary_logloss: 0.000930295
[800]	valid_0's binary_logloss: 0.000929445
[801]	valid_0's binary_logloss: 0.000928614
[802]	valid_0's binary_logloss: 0.000927731
[803]	valid_0's binary_logloss: 0.

...

[2085]	valid_0's binary_logloss: 0.000478923
[2086]	valid_0's binary_logloss: 0.000478762
[2087]	valid_0's binary_logloss: 0.000478601
[2088]	valid_0's binary_logloss: 0.00047844
[2089]	valid_0's binary_logloss: 0.000478281
[2090]	valid_0's binary_logloss: 0.000478212
[2091]	valid_0's binary_logloss: 0.000478154
[2092]	valid_0's binary_logloss: 0.000478088
[2093]	valid_0's binary_logloss: 0.000478031
[2094]	valid_0's binary_logloss: 0.00047794
[2095]	valid_0's binary_logloss: 0.000477883
[2096]	valid_0's binary_logloss: 0.000477725
[2097]	valid_0's binary_logloss: 0.000477568
[2098]	valid_0's binary_logloss: 0.000477336
[2099]	valid_0's binary_logloss: 0.000477179
[2100]	valid_0's binary_logloss: 0.000477066
[2101]	valid_0's binary_logloss: 0.000476953
[2102]	valid_0's binary_logloss: 0.000476777
[2103]	valid_0's binary_logloss: 0.000476714
[2104]	valid_0's binary_logloss: 0.000476602
[2105]	valid_0's binary_logloss: 0.00047649
[2106]	valid_0's binary_logloss: 0.000476412
[2107]	valid_

[2269]	valid_0's binary_logloss: 0.000459733
[2270]	valid_0's binary_logloss: 0.00045968
[2271]	valid_0's binary_logloss: 0.000459585
[2272]	valid_0's binary_logloss: 0.000459493
[2273]	valid_0's binary_logloss: 0.000459398
[2274]	valid_0's binary_logloss: 0.000459303
[2275]	valid_0's binary_logloss: 0.00045921
[2276]	valid_0's binary_logloss: 0.000459086
[2277]	valid_0's binary_logloss: 0.000459069
[2278]	valid_0's binary_logloss: 0.000459052
[2279]	valid_0's binary_logloss: 0.000458929
[2280]	valid_0's binary_logloss: 0.000458806
[2281]	valid_0's binary_logloss: 0.00045875
[2282]	valid_0's binary_logloss: 0.000458694
[2283]	valid_0's binary_logloss: 0.000458638
[2284]	valid_0's binary_logloss: 0.000458583
[2285]	valid_0's binary_logloss: 0.000458493
[2286]	valid_0's binary_logloss: 0.000458494
[2287]	valid_0's binary_logloss: 0.000458496
[2288]	valid_0's binary_logloss: 0.000458498
[2289]	valid_0's binary_logloss: 0.000458501
[2290]	valid_0's binary_logloss: 0.000458503
[2291]	valid_

[2452]	valid_0's binary_logloss: 0.000443561
[2453]	valid_0's binary_logloss: 0.000443504
[2454]	valid_0's binary_logloss: 0.000443405
[2455]	valid_0's binary_logloss: 0.000443189
[2456]	valid_0's binary_logloss: 0.000443161
[2457]	valid_0's binary_logloss: 0.000443132
[2458]	valid_0's binary_logloss: 0.000443104
[2459]	valid_0's binary_logloss: 0.000443027
[2460]	valid_0's binary_logloss: 0.000442999
[2461]	valid_0's binary_logloss: 0.000442915
[2462]	valid_0's binary_logloss: 0.000442765
[2463]	valid_0's binary_logloss: 0.000442616
[2464]	valid_0's binary_logloss: 0.000442466
[2465]	valid_0's binary_logloss: 0.000442318
[2466]	valid_0's binary_logloss: 0.000442358
[2467]	valid_0's binary_logloss: 0.000442398
[2468]	valid_0's binary_logloss: 0.000442439
[2469]	valid_0's binary_logloss: 0.000442505
[2470]	valid_0's binary_logloss: 0.000442546
[2471]	valid_0's binary_logloss: 0.000442501
[2472]	valid_0's binary_logloss: 0.000442487
[2473]	valid_0's binary_logloss: 0.000442393
[2474]	val

[2641]	valid_0's binary_logloss: 0.000432134
[2642]	valid_0's binary_logloss: 0.00043208
[2643]	valid_0's binary_logloss: 0.000431966
[2644]	valid_0's binary_logloss: 0.000431897
[2645]	valid_0's binary_logloss: 0.000431843
[2646]	valid_0's binary_logloss: 0.000431797
[2647]	valid_0's binary_logloss: 0.000431784
[2648]	valid_0's binary_logloss: 0.000431738
[2649]	valid_0's binary_logloss: 0.000431693
[2650]	valid_0's binary_logloss: 0.000431647
[2651]	valid_0's binary_logloss: 0.000431515
[2652]	valid_0's binary_logloss: 0.000431383
[2653]	valid_0's binary_logloss: 0.000431351
[2654]	valid_0's binary_logloss: 0.000431205
[2655]	valid_0's binary_logloss: 0.000431073
[2656]	valid_0's binary_logloss: 0.000431111
[2657]	valid_0's binary_logloss: 0.000431086
[2658]	valid_0's binary_logloss: 0.000431126
[2659]	valid_0's binary_logloss: 0.000431163
[2660]	valid_0's binary_logloss: 0.000431201
[2661]	valid_0's binary_logloss: 0.000431159
[2662]	valid_0's binary_logloss: 0.000431118
[2663]	vali

[2828]	valid_0's binary_logloss: 0.000425981
[2829]	valid_0's binary_logloss: 0.000425938
[2830]	valid_0's binary_logloss: 0.000425924
[2831]	valid_0's binary_logloss: 0.000425874
[2832]	valid_0's binary_logloss: 0.000425823
[2833]	valid_0's binary_logloss: 0.000425747
[2834]	valid_0's binary_logloss: 0.000425697
[2835]	valid_0's binary_logloss: 0.000425665
[2836]	valid_0's binary_logloss: 0.000425652
[2837]	valid_0's binary_logloss: 0.000425667
[2838]	valid_0's binary_logloss: 0.000425675
[2839]	valid_0's binary_logloss: 0.000425684
[2840]	valid_0's binary_logloss: 0.000425693
[2841]	valid_0's binary_logloss: 0.000425735
[2842]	valid_0's binary_logloss: 0.000425778
[2843]	valid_0's binary_logloss: 0.000425831
[2844]	valid_0's binary_logloss: 0.000425874
[2845]	valid_0's binary_logloss: 0.000425872
[2846]	valid_0's binary_logloss: 0.000425911
[2847]	valid_0's binary_logloss: 0.000425874
[2848]	valid_0's binary_logloss: 0.000425834
[2849]	valid_0's binary_logloss: 0.00042579
[2850]	vali

In [38]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

feature_imp = pd.DataFrame(sorted(zip(gbm.feature_importance(),X_test.columns)), columns=['Value','Feature'])
feature_imp

,Value,Feature
0,0,img_count
1,0,pred2_hare
2,0,xepred_porcupine
3,2,pred2_aardvark
4,2,pred2_mongoose
5,2,pred2_wildcat
6,2,pred_porcupine
7,2,xepred2_aardwolf
8,2,xepred2_hyenastriped
9,2,xepred2_zorilla


In [40]:
for clazz in list(train_labels.columns):
    X_train = train_dataset[columns]
    y_train = train_dataset[clazz]
    if sum(y_train) < 5:
        continue
    X_val = val_dataset[columns]
    y_val = val_dataset[clazz]

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

    scale = y_train.mean()
    if scale < 0.5:
        scale=math.log(1/scale)
    else:
        scale=math.sqrt(scale)
    print(scale)
    params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': {'binary_logloss'},
            'num_leaves': num_leaves.get(clazz, 5),
            'max_depth': max_depth.get(clazz, 3),
            'learning_rate': learning_rate.get(clazz, 0.01/scale),
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0,
    #         'scale_pos_weight':scale_pos_weight.get(clazz, scale)
            'scale_pos_weight':1

    }


    gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round=10000,
                        valid_sets=lgb_val,
                        early_stopping_rounds=50)
    gbm.save_model("lgbs_v3/" + clazz + '.txt', num_iteration=gbm.best_iteration) 

    X_test = test_dataset[columns]
    y_test = test_dataset[clazz]
    if sum(y_test) ==0:
        print(clazz)
    else:
        print(clazz, log_loss(y_test, gbm.predict(X_test.values)))

8.144665360696893
[1]	valid_0's binary_logloss: 0.00179573
Training until validation scores don't improve for 50 rounds
[2]	valid_0's binary_logloss: 0.00177332
[3]	valid_0's binary_logloss: 0.00174403
[4]	valid_0's binary_logloss: 0.00171304
[5]	valid_0's binary_logloss: 0.00168645
[6]	valid_0's binary_logloss: 0.00166685
[7]	valid_0's binary_logloss: 0.00165279
[8]	valid_0's binary_logloss: 0.00163687
[9]	valid_0's binary_logloss: 0.00162497
[10]	valid_0's binary_logloss: 0.00161245
[11]	valid_0's binary_logloss: 0.00159436
[12]	valid_0's binary_logloss: 0.00157853
[13]	valid_0's binary_logloss: 0.00156756
[14]	valid_0's binary_logloss: 0.00155379
[15]	valid_0's binary_logloss: 0.0015424
[16]	valid_0's binary_logloss: 0.00153011
[17]	valid_0's binary_logloss: 0.00151846
[18]	valid_0's binary_logloss: 0.00150365
[19]	valid_0's binary_logloss: 0.00149443
[20]	valid_0's binary_logloss: 0.00148575
[21]	valid_0's binary_logloss: 0.00147806
[22]	valid_0's binary_logloss: 0.0014706
[23]	val

[194]	valid_0's binary_logloss: 0.000955535
[195]	valid_0's binary_logloss: 0.000954587
[196]	valid_0's binary_logloss: 0.000953073
[197]	valid_0's binary_logloss: 0.000951951
[198]	valid_0's binary_logloss: 0.000950445
[199]	valid_0's binary_logloss: 0.000948951
[200]	valid_0's binary_logloss: 0.000947463
[201]	valid_0's binary_logloss: 0.000946092
[202]	valid_0's binary_logloss: 0.000944801
[203]	valid_0's binary_logloss: 0.000943338
[204]	valid_0's binary_logloss: 0.000942043
[205]	valid_0's binary_logloss: 0.000940754
[206]	valid_0's binary_logloss: 0.000939501
[207]	valid_0's binary_logloss: 0.000937934
[208]	valid_0's binary_logloss: 0.000936694
[209]	valid_0's binary_logloss: 0.000935459
[210]	valid_0's binary_logloss: 0.00093423
[211]	valid_0's binary_logloss: 0.00093279
[212]	valid_0's binary_logloss: 0.000931503
[213]	valid_0's binary_logloss: 0.000930119
[214]	valid_0's binary_logloss: 0.000928694
[215]	valid_0's binary_logloss: 0.000927423
[216]	valid_0's binary_logloss: 0.

[381]	valid_0's binary_logloss: 0.00077167
[382]	valid_0's binary_logloss: 0.000770922
[383]	valid_0's binary_logloss: 0.000770166
[384]	valid_0's binary_logloss: 0.00076933
[385]	valid_0's binary_logloss: 0.000768571
[386]	valid_0's binary_logloss: 0.000767891
[387]	valid_0's binary_logloss: 0.000767213
[388]	valid_0's binary_logloss: 0.000766605
[389]	valid_0's binary_logloss: 0.00076593
[390]	valid_0's binary_logloss: 0.000765257
[391]	valid_0's binary_logloss: 0.000764495
[392]	valid_0's binary_logloss: 0.000763735
[393]	valid_0's binary_logloss: 0.000762976
[394]	valid_0's binary_logloss: 0.00076222
[395]	valid_0's binary_logloss: 0.000761466
[396]	valid_0's binary_logloss: 0.000760709
[397]	valid_0's binary_logloss: 0.00075985
[398]	valid_0's binary_logloss: 0.000759099
[399]	valid_0's binary_logloss: 0.000758383
[400]	valid_0's binary_logloss: 0.000757634
[401]	valid_0's binary_logloss: 0.000757073
[402]	valid_0's binary_logloss: 0.00075642
[403]	valid_0's binary_logloss: 0.0007

[571]	valid_0's binary_logloss: 0.000656066
[572]	valid_0's binary_logloss: 0.000655546
[573]	valid_0's binary_logloss: 0.000655155
[574]	valid_0's binary_logloss: 0.000654765
[575]	valid_0's binary_logloss: 0.000654376
[576]	valid_0's binary_logloss: 0.000653907
[577]	valid_0's binary_logloss: 0.000653439
[578]	valid_0's binary_logloss: 0.000652868
[579]	valid_0's binary_logloss: 0.000652402
[580]	valid_0's binary_logloss: 0.000651937
[581]	valid_0's binary_logloss: 0.000651438
[582]	valid_0's binary_logloss: 0.000650855
[583]	valid_0's binary_logloss: 0.000650358
[584]	valid_0's binary_logloss: 0.000649862
[585]	valid_0's binary_logloss: 0.000649367
[586]	valid_0's binary_logloss: 0.000649099
[587]	valid_0's binary_logloss: 0.000648832
[588]	valid_0's binary_logloss: 0.000648566
[589]	valid_0's binary_logloss: 0.000648301
[590]	valid_0's binary_logloss: 0.000648037
[591]	valid_0's binary_logloss: 0.000647592
[592]	valid_0's binary_logloss: 0.000647148
[593]	valid_0's binary_logloss: 

[758]	valid_0's binary_logloss: 0.000577314
[759]	valid_0's binary_logloss: 0.000576937
[760]	valid_0's binary_logloss: 0.000576625
[761]	valid_0's binary_logloss: 0.000576245
[762]	valid_0's binary_logloss: 0.000575865
[763]	valid_0's binary_logloss: 0.000575486
[764]	valid_0's binary_logloss: 0.000575108
[765]	valid_0's binary_logloss: 0.000574731
[766]	valid_0's binary_logloss: 0.000574361
[767]	valid_0's binary_logloss: 0.000573991
[768]	valid_0's binary_logloss: 0.000573622
[769]	valid_0's binary_logloss: 0.000573254
[770]	valid_0's binary_logloss: 0.000572837
[771]	valid_0's binary_logloss: 0.000572438
[772]	valid_0's binary_logloss: 0.000572058
[773]	valid_0's binary_logloss: 0.00057163
[774]	valid_0's binary_logloss: 0.000571234
[775]	valid_0's binary_logloss: 0.000570838
[776]	valid_0's binary_logloss: 0.000570467
[777]	valid_0's binary_logloss: 0.000570097
[778]	valid_0's binary_logloss: 0.000569728
[779]	valid_0's binary_logloss: 0.000569359
[780]	valid_0's binary_logloss: 0

...

[985]	valid_0's binary_logloss: 0.000575597
[986]	valid_0's binary_logloss: 0.000575262
[987]	valid_0's binary_logloss: 0.000574947
[988]	valid_0's binary_logloss: 0.000574561
[989]	valid_0's binary_logloss: 0.000574267
[990]	valid_0's binary_logloss: 0.000573954
[991]	valid_0's binary_logloss: 0.000573643
[992]	valid_0's binary_logloss: 0.000573331
[993]	valid_0's binary_logloss: 0.000572999
[994]	valid_0's binary_logloss: 0.000572688
[995]	valid_0's binary_logloss: 0.000572334
[996]	valid_0's binary_logloss: 0.000572067
[997]	valid_0's binary_logloss: 0.000571637
[998]	valid_0's binary_logloss: 0.000571371
[999]	valid_0's binary_logloss: 0.000571105
[1000]	valid_0's binary_logloss: 0.000570841
[1001]	valid_0's binary_logloss: 0.000570397
[1002]	valid_0's binary_logloss: 0.000569997
[1003]	valid_0's binary_logloss: 0.000569598
[1004]	valid_0's binary_logloss: 0.000569157
[1005]	valid_0's binary_logloss: 0.000568717
[1006]	valid_0's binary_logloss: 0.000568502
[1007]	valid_0's binary_l

[1170]	valid_0's binary_logloss: 0.000520475
[1171]	valid_0's binary_logloss: 0.000520106
[1172]	valid_0's binary_logloss: 0.000519739
[1173]	valid_0's binary_logloss: 0.000519372
[1174]	valid_0's binary_logloss: 0.000519062
[1175]	valid_0's binary_logloss: 0.000518697
[1176]	valid_0's binary_logloss: 0.000518437
[1177]	valid_0's binary_logloss: 0.000518178
[1178]	valid_0's binary_logloss: 0.000517919
[1179]	valid_0's binary_logloss: 0.000517658
[1180]	valid_0's binary_logloss: 0.0005174
[1181]	valid_0's binary_logloss: 0.000517041
[1182]	valid_0's binary_logloss: 0.000516821
[1183]	valid_0's binary_logloss: 0.0005166
[1184]	valid_0's binary_logloss: 0.000516381
[1185]	valid_0's binary_logloss: 0.000516161
[1186]	valid_0's binary_logloss: 0.000515904
[1187]	valid_0's binary_logloss: 0.000515691
[1188]	valid_0's binary_logloss: 0.000515435
[1189]	valid_0's binary_logloss: 0.000515179
[1190]	valid_0's binary_logloss: 0.000514924
[1191]	valid_0's binary_logloss: 0.000514694
[1192]	valid_0

[1357]	valid_0's binary_logloss: 0.00047751
[1358]	valid_0's binary_logloss: 0.000477339
[1359]	valid_0's binary_logloss: 0.00047708
[1360]	valid_0's binary_logloss: 0.00047682
[1361]	valid_0's binary_logloss: 0.00047666
[1362]	valid_0's binary_logloss: 0.000476501
[1363]	valid_0's binary_logloss: 0.000476309
[1364]	valid_0's binary_logloss: 0.000476098
[1365]	valid_0's binary_logloss: 0.000475921
[1366]	valid_0's binary_logloss: 0.000475657
[1367]	valid_0's binary_logloss: 0.00047544
[1368]	valid_0's binary_logloss: 0.000475223
[1369]	valid_0's binary_logloss: 0.00047501
[1370]	valid_0's binary_logloss: 0.000474794
[1371]	valid_0's binary_logloss: 0.000474527
[1372]	valid_0's binary_logloss: 0.00047426
[1373]	valid_0's binary_logloss: 0.000473993
[1374]	valid_0's binary_logloss: 0.000473727
[1375]	valid_0's binary_logloss: 0.000473461
[1376]	valid_0's binary_logloss: 0.000473156
[1377]	valid_0's binary_logloss: 0.000472852
[1378]	valid_0's binary_logloss: 0.000472549
[1379]	valid_0's 

[1540]	valid_0's binary_logloss: 0.000445683
[1541]	valid_0's binary_logloss: 0.000445559
[1542]	valid_0's binary_logloss: 0.000445436
[1543]	valid_0's binary_logloss: 0.000445263
[1544]	valid_0's binary_logloss: 0.00044514
[1545]	valid_0's binary_logloss: 0.000445018
[1546]	valid_0's binary_logloss: 0.000444883
[1547]	valid_0's binary_logloss: 0.000444748
[1548]	valid_0's binary_logloss: 0.000444613
[1549]	valid_0's binary_logloss: 0.000444546
[1550]	valid_0's binary_logloss: 0.00044448
[1551]	valid_0's binary_logloss: 0.000444336
[1552]	valid_0's binary_logloss: 0.000444137
[1553]	valid_0's binary_logloss: 0.000443899
[1554]	valid_0's binary_logloss: 0.000443662
[1555]	valid_0's binary_logloss: 0.000443424
[1556]	valid_0's binary_logloss: 0.000443299
[1557]	valid_0's binary_logloss: 0.000443173
[1558]	valid_0's binary_logloss: 0.000443048
[1559]	valid_0's binary_logloss: 0.000442923
[1560]	valid_0's binary_logloss: 0.000442798
[1561]	valid_0's binary_logloss: 0.000442686
[1562]	valid

KeyboardInterrupt: 

In [ ]:
X_test = test_dataset[columns]
ls = 0
c = list(train_labels.columns)
c.remove('reptiles')
for clazz in c:
    y_test = test_dataset[clazz]
    if sum(y_test) ==0 :
        print(clazz, 'bad')
        continue
    model = lgb.Booster(model_file="lgbs_v3/" + clazz + '.txt')
    l =log_loss(y_test, model.predict(X_test))
    ls+=l
    print(clazz, l)
print(ls/54)

In [ ]:
aardvark 0.00014215543184568048
aardwolf 0.0004508591364472872
baboon 0.0017464964936849687
batearedfox 0.0005584773089463318
buffalo 0.004205705114248883
bushbuck 0.00013461444133176055
caracal 0.00012774976768483492
cheetah 0.0014921033821730734
civet 3.1596233544084036e-05
dikdik 0.001792085498978284
duiker 0.0008978360513132168
eland 0.001427325241307141
elephant 0.004358624955669968
empty 0.05526157837202803
gazellegrants 0.007334518590201408
gazellethomsons 0.016744181232038757
genet 0.00029686300661493925
giraffe 0.0027515076466454987
guineafowl 0.0013640892852717966
hare 0.0004629380564604658
hartebeest 0.0028042625995350116
hippopotamus 0.0006695931172353393
honeybadger 0.00010805832232163028
hyenaspotted 0.002764638317896244
hyenastriped 0.00021769289634456898
impala 0.004805353971731504
insectspider 0.003823473395375832
jackal 0.0010280228448581578
koribustard 0.0009780002268577643
leopard 0.00041891385042394153
lionfemale 0.002135840086380835
lionmale 0.0012224710031886828
mongoose 0.0003039907487659614
monkeyvervet 0.0005216045752840155
ostrich 0.0014244204261452426
otherbird 0.01330519284233979
porcupine 8.809166204356199e-05
reedbuck 0.0016403559146369786
reptiles 0.0011768364366852716
secretarybird 0.001088329233431082
serval 0.0003103944567042552
steenbok 0.00043182499834808815
topi 0.002411482378446688
vulture 0.0001957507678821575
warthog 0.004235989021126352
waterbuck 8.321889807911485e-05
wildebeest 0.01914542572062037
zebra 0.017865518797304348
0.0034590009769709116


In [61]:
list(train_labels.columns)

['aardvark',
 'aardwolf',
 'baboon',
 'bat',
 'batearedfox',
 'buffalo',
 'bushbuck',
 'caracal',
 'cattle',
 'cheetah',
 'civet',
 'dikdik',
 'duiker',
 'eland',
 'elephant',
 'empty',
 'gazellegrants',
 'gazellethomsons',
 'genet',
 'giraffe',
 'guineafowl',
 'hare',
 'hartebeest',
 'hippopotamus',
 'honeybadger',
 'hyenaspotted',
 'hyenastriped',
 'impala',
 'insectspider',
 'jackal',
 'koribustard',
 'leopard',
 'lionfemale',
 'lionmale',
 'mongoose',
 'monkeyvervet',
 'ostrich',
 'otherbird',
 'porcupine',
 'reedbuck',
 'reptiles',
 'rhinoceros',
 'rodents',
 'secretarybird',
 'serval',
 'steenbok',
 'topi',
 'vulture',
 'warthog',
 'waterbuck',
 'wildcat',
 'wildebeest',
 'zebra',
 'zorilla']